In [1]:
import pandas as pd
import seaborn as sb
import math
import numpy as np
import os
import re
import jenkspy

import postProcessing
import matplotlib.pyplot as plt

In [2]:
#Need to run only once
#df = pd.read_csv("MetaFeatures.csv")

#temp = []
#for i in df['LabelIssues']:
 #   if(i == '[]'):
 #       temp.append(0)
  #  else:
   #     temp.append(1)

#df['LabelIssuesPerc']  = temp
#df['LabelIssuesPerc'].value_counts()

#df.to_csv("df_mfeatures.csv",index=False)

In [3]:
df1 = pd.read_csv("df_mfeatures.csv")
df2 = pd.read_csv("df_ensemble_allfeatures.csv")

In [4]:
df1.columns

Index(['File', 'Completeness', 'MissColumns', 'Conciseness', 'SyntaxAccuracy',
       'InvalidColumns', 'cor.mean', 'cor.sd', 'cov.mean', 'cov.sd',
       'eigenvalues.mean', 'eigenvalues.sd', 'g_mean.mean', 'g_mean.sd',
       'h_mean.mean', 'h_mean.sd', 'iq_range.mean', 'iq_range.sd',
       'kurtosis.mean', 'kurtosis.sd', 'mad.mean', 'mad.sd', 'max.mean',
       'max.sd', 'mean.mean', 'mean.sd', 'median.mean', 'median.sd',
       'min.mean', 'min.sd', 'nr_cor_attr', 'nr_norm', 'nr_outliers',
       'range.mean', 'range.sd', 'sd.mean', 'sd.sd', 'skewness.mean',
       'skewness.sd', 'sparsity.mean', 'sparsity.sd', 't_mean.mean',
       't_mean.sd', 'var.mean', 'var.sd', 'ClassImbPoints', 'ClassImbRatio',
       'ClassOverlapPoints', 'ClassOverlapPerc', 'OutlierPoints',
       'OutlierPerc', 'LabelIssues', 'attr_to_inst', 'cat_to_num',
       'inst_to_attr', 'nr_attr', 'nr_bin', 'nr_cat', 'nr_inst', 'nr_num',
       'num_to_cat', 'attr_conc.mean', 'attr_conc.sd', 'attr_ent.mean',
    

In [5]:
df2.columns

Index(['File', 'FeatureAlgo', 'Accuracy', 'Precision macro', 'Recall macro',
       'Precision weight', 'Recall weight', 'Time FS', 'Features',
       'Time ensemble', 'class index'],
      dtype='object')

## Syntax accuracy is 0 for all data points, as datasets has same valid column types. Hence, dropped

In [6]:
bin_df, dict_values = postProcessing.generate_categories(df1)
bin_df.to_csv("BinnedMetaFeatures.csv",index=False)

In [7]:
#creating limits for each metric
item_limits = {}
features = []
for item in dict_values.items():
  temp = []
  temp.append(item[0])
  temp.append(item[1][0])
  temp.append(item[1][1])
  temp.append(item[1][1])
  temp.append(item[1][2])
  features.append(temp)

df_new = pd.DataFrame(features)

df_new.columns = ['features','small_lower','small_upper','big_lower','big_upper']
df_new.set_index('features')
df_new.to_csv("BinLimits.csv",index=False)

## Creating normalised values

In [1]:
import numpy as np
import pandas as pd

def check_continuous_and_normalize(df):
    df = df.copy()
    
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]) and df[column].nunique() > 20:
            mean = df[column].mean()
            std = df[column].std()
            new_column_name = f"{column}_normalized"
            df[new_column_name] = np.where((df[column] >= mean - std) & (df[column] <= mean + std), 1, 0)
        else:
            print("column",column)
            new_column_name = f"{column}_normalized"
            df[new_column_name] = df[column]

    df.drop(columns=df.columns.difference([col for col in df.columns if '_normalized' in col]), inplace=True)
    return df

In [2]:
df = pd.read_csv("KnowledgeBase.csv")
df1 =  df[['Completeness', 'Conciseness', 'cor.mean', 'cov.mean', 'eigenvalues.mean', 'g_mean.mean', 'h_mean.mean', 'iq_range.mean', 
 'kurtosis.mean', 'mad.mean', 'max.mean', 'mean.mean', 'median.mean', 'min.mean', 'nr_cor_attr', 'nr_norm', 'nr_outliers', 'range.mean', 'sd.mean', 'skewness.mean', 'sparsity.mean',
 't_mean.mean', 'var.mean', 'ClassImbRatio', 'ClassOverlapPerc', 'OutlierPerc', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin', 'nr_inst', 'nr_num','attr_conc.mean', 
 'attr_ent.mean', 'LabelIssuesPerc','nUnique', 'ena', 'snr.mean', 'cEntropy', 'FeatureAlgo']]

df_normalised = check_continuous_and_normalize(df1)

column nr_norm
column ClassImbRatio
column nUnique
column FeatureAlgo


In [12]:
df_normalised.to_csv("NormalisedDataset.csv",index=False)

In [13]:
def calculate_statistics(df):
    statistics = []

    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]) and df[column].nunique() > 20:
            mean = df[column].mean()
            std = df[column].std()
            statistics.append({'Column': column, 'Mean': mean, 'Standard Deviation': std})
    return pd.DataFrame(statistics)


statistics_df = calculate_statistics(df1)

statistics_df.to_csv('NormalizationValues.csv', index=False)